In [8]:
import numpy as np
import pandas as pd
import json
import os
from tabulate import tabulate

In [9]:
folder_path = r"C:\Users\Johnk\Documents\PhD\Quantum Computing Code\Quantum-Computing\SUSY\SUSY QM\PennyLane\VQE\Differential Evolution\RealAmplitudes\VaryShots"
dataframes = {}
folders = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]
folders

['1024', '16384', '256', '4096', '64', '65536']

In [10]:
converged_runs = []
mean_iters = []
mean_evals = []
min_e = []
delta_min_e = []
median_e = []
delta_median_e = []
exact_e = []
time_taken = []

num_vqe = []
shots_list = []
maxiter = []

cutoff = 16
potential = 'DW'
shot_list = [64, 256, 1024, 4096, 16384, 65536]


for folder in folders:

    d_path = os.path.join(folder_path, folder, f'{potential}_{cutoff}.json')

    with open(d_path, 'r') as json_file:
        data = json.load(json_file)

    converged_indices = [i for i, success in enumerate(data['success']) if success]

    num_vqe.append(data['num_VQE'])
    shots_list.append(data['shots'])

    optimzer = data['Optimizer']
    maxiter.append(optimzer['maxiter'])
    
    exact_min_e = np.min(data['exact_eigenvalues'])
    exact_e.append(exact_min_e)

    converged_runs.append(sum(data['success']))

    #iters = [data['num_iters'][i] for i in converged_indices]
    #mean_iters.append(int(np.round(np.mean(iters))))
    mean_iters.append(int(np.round(np.mean(data['num_iters']))))

    #evals = [data['num_evaluations'][i] for i in converged_indices]
    #mean_evals.append(int(np.round(np.mean(evals))))
    mean_evals.append(int(np.round(np.mean(data['num_evaluations']))))

    results = [data['results'][i] for i in converged_indices]
    min_e.append(np.min(results))
    delta_min_e.append(abs(exact_min_e - np.min(results)))
    median_e.append(np.median(results))
    delta_median_e.append(abs(exact_min_e - np.median(results)))

    #run_times = [data['run_times'][i] for i in converged_indices]
    #time_series = np.array([pd.Timedelta(t).total_seconds() for t in run_times])
    time_series = np.array([pd.Timedelta(t).total_seconds() for t in data['run_times']])
    mean_vqe_time = np.mean(time_series)
    #mean_vqe_time = pd.Timedelta(seconds=np.mean(time_series))
    time_taken.append(np.mean(mean_vqe_time))
    #time_taken.append(data['parallel_run_time'])

In [16]:
df = pd.DataFrame({
    #'potential': potential,
    #'cutoff': cutoff,
    'shots': shots_list,
    'Converged Runs': [f"{c}/100" for c in converged_runs],
    'Mean Iter': mean_iters,
    'Mean evals': mean_evals,
    'Mean VQE time (s)': [f"{t:.2f}" for t in time_taken],
    'E_VQE': [f"{num:.5e}" for num in median_e],
    'Exact': [f"{num:.5e}" for num in exact_e],
    'Delta': [f"{num:.5e}" for num in delta_median_e],
    #'VQE min E': [f"{num:.5e}" for num in min_e],
    #'Delta min E': [f"{num:.5e}" for num in delta_min_e],
    #'VQE median E': [f"{num:.5e}" for num in median_e],
    #'Delta median E': [f"{num:.5e}" for num in delta_median_e],
    
})

In [17]:
df['shots'] = df['shots'].astype(int)
df = df.sort_values('shots')
df

,shots,Converged Runs,Mean Iter,Mean evals,Mean VQE time (s),E_VQE,Exact,Delta
4,64,100/100,298,6250,23.74,8.91633e-01,8.91599e-01,3.34617e-05
2,256,100/100,264,5581,24.95,8.91639e-01,8.91599e-01,4.01215e-05
0,1024,100/100,237,5037,22.00,8.91641e-01,8.91599e-01,4.13398e-05
3,4096,100/100,237,5044,21.33,8.91641e-01,8.91599e-01,4.15733e-05
1,16384,100/100,187,4039,18.12,8.93297e-01,8.91599e-01,1.69764e-03
5,65536,100/100,151,3330,23.62,8.94868e-01,8.91599e-01,3.26878e-03


In [26]:
latex_table = tabulate(df.values, headers=df.columns, tablefmt="latex", floatfmt=".5g")
print(latex_table)

\begin{tabular}{rlrrrrrr}
\hline
   shots & Converged Runs   &   Mean Iter &   Mean evals &   Mean VQE time (s) &   E\_VQE &   Exact &      Delta \\
\hline
      64 & 100/100          &         298 &         6250 &               23.74 & 0.89163 &  0.8916 & 3.3462e-05 \\
     256 & 100/100          &         264 &         5581 &               24.95 & 0.89164 &  0.8916 & 4.0121e-05 \\
    1024 & 100/100          &         237 &         5037 &               22    & 0.89164 &  0.8916 & 4.134e-05  \\
    4096 & 100/100          &         237 &         5044 &               21.33 & 0.89164 &  0.8916 & 4.1573e-05 \\
   16384 & 100/100          &         187 &         4039 &               18.12 & 0.8933  &  0.8916 & 0.0016976  \\
   65536 & 100/100          &         151 &         3330 &               23.62 & 0.89487 &  0.8916 & 0.0032688  \\
\hline
\end{tabular}


In [30]:
df

,potential,cutoff,Converged Runs,Mean Iter,Exact,VQE min E,Delta min E,VQE median E,Delta median E,Time Taken
0,AHO,2,100/100,7,-4.37500e-01,-4.37500e-01,0.00000e+00,-4.37500e-01,0.00000e+00,0:00:01.464709
1,AHO,4,100/100,9,-1.64785e-01,-1.64785e-01,4.99600e-16,-1.64785e-01,4.99600e-16,0:00:01.847212
2,AHO,8,100/100,50,3.20101e-02,3.20101e-02,1.26218e-14,3.20101e-02,1.26218e-14,0:00:10.802894
3,AHO,16,100/100,313,-1.16698e-03,-1.16698e-03,4.05626e-13,-1.16698e-03,4.05626e-13,0:04:02.523365
4,AHO,32,100/100,214,6.18224e-06,6.18224e-06,3.17674e-12,6.18224e-06,3.17674e-12,0:01:53.709032
5,AHO,64,99/100,377,-1.94209e-08,-1.94478e-08,2.68587e-11,-1.94478e-08,2.68587e-11,0:19:31.655587
6,AHO,128,100/100,241,-1.98015e-11,1.40750e-12,2.12090e-11,1.40750e-12,2.12090e-11,0:09:34.818436
7,AHO,256,100/100,198,1.49740e-10,2.15004e-11,1.28240e-10,2.15004e-11,1.28240e-10,0:53:43.076241
8,AHO,512,100/100,230,-3.80617e-10,7.42043e-11,4.54821e-10,7.42043e-11,4.54821e-10,4:56:52.321489
9,DW,2,100/100,7,3.57233e-01,3.57233e-01,5.55112e-17,3.57233e-01,5.55112e-17,0:00:01.520095


In [31]:
latex_table = tabulate(df.values, headers=df.columns, tablefmt="latex")#, floatfmt=".3f")
print(latex_table)


\begin{tabular}{lrlrrrrrrl}
\hline
 potential   &   cutoff & Converged Runs   &   Mean Iter &        Exact &    VQE min E &   Delta min E &   VQE median E &   Delta median E & Time Taken      \\
\hline
 AHO         &        2 & 100/100          &           7 & -0.4375      & -0.4375      &   0           &   -0.4375      &      0           & 0:00:01.464709  \\
 AHO         &        4 & 100/100          &           9 & -0.164785    & -0.164785    &   4.996e-16   &   -0.164785    &      4.996e-16   & 0:00:01.847212  \\
 AHO         &        8 & 100/100          &          50 &  0.0320101   &  0.0320101   &   1.26218e-14 &    0.0320101   &      1.26218e-14 & 0:00:10.802894  \\
 AHO         &       16 & 100/100          &         313 & -0.00116698  & -0.00116698  &   4.05626e-13 &   -0.00116698  &      4.05626e-13 & 0:04:02.523365  \\
 AHO         &       32 & 100/100          &         214 &  6.18224e-06 &  6.18224e-06 &   3.17674e-12 &    6.18224e-06 &      3.17674e-12 & 0:01:53.709032  \